In [1]:
import pandas as pd
import json
import numpy as np

In [2]:
building_data_csv = 'Building data sampling - Data.csv'
bldg_df = pd.read_csv(building_data_csv, header=0)

annotations_json = 'mapillary_annotations.json'
with open(annotations_json, 'r') as f:
    anno_coco = json.load(f)

mapping_csv = 'log_mapillary.txt'
mapping_df = pd.read_csv(mapping_csv, header=0)

In [3]:
attrib_mapping = {
    'Building category': 'category',
    'Occupancy group': 'occupancy_group',
    'Occupancy type': 'occupancy_type',
    'Number of floors (excl. basement)': 'no_floors',
    'Basement':'basement',
    'Material':'material',
    'Roof shape':'roof_shape',
    'Roof covers':'roof_covers',
    'Shutters':'shutters',
    'Window area':'window_area'
}

In [4]:
# add missing building category attribute to json

# for anno in anno_coco['annotations']:
#     anno['attributes']['building_category'] = 'Unknown'

In [4]:
# based on mapping data, add the attribute data from csv into COCO annotations

for index, (image_id, bbox_id) in mapping_df.iterrows():
    # retrieve row from bldg_df based on image_id
    attrib_data = bldg_df.iloc[image_id,:]
    
    # retrieve anno data based on bbox_id
    attrib_anno = anno_coco['annotations'][bbox_id-1]['attributes']
    
    # make the data match the annotation
    for data_key, anno_key in attrib_mapping.items():
        # print("Processing:", attrib_data.loc[data_key])
        if isinstance(attrib_data.loc[data_key], float) and np.isnan(attrib_data.loc[data_key]):
            attrib_anno[anno_key] = 0
            continue
        if anno_key == "occupancy_group" and (attrib_data.loc[data_key] != "Unknown" or attrib_data.loc[data_key] != "Other"):
            # example: 'RES: Residential'
            
            attrib_anno[anno_key] = attrib_data.loc[data_key].split(":")[0]
        elif anno_key == "occupancy_type" and attrib_data.loc[data_key] != "Unknown":
#             if isinstance(attrib_data.loc[data_key],float):
#                 print("IM A FLOAT")
                
#                 print(np.isnan(attrib_data.loc[data_key]))
                
#             print(type(attrib_data.loc[data_key]))
#             print(isinstance(attrib_data.loc[data_key],float))
            
            
            # example 'RES1: Single Family Home'
            attrib_anno[anno_key] = attrib_data.loc[data_key].split(":")[0]
        else:
            attrib_anno[anno_key] = attrib_data.loc[data_key]

In [5]:
# fix image id in images

old_new_id_mapping = {}

for image in anno_coco['images']:
    # example file_name: image_684.png
    old_id = image['id']
    new_id = int(image['file_name'].split('_')[1].split('.')[0])
    old_new_id_mapping[old_id] = new_id
    image['id'] = new_id

# fix image_id in annotation

for anno in anno_coco['annotations']:
    old_id = anno['image_id']
    anno['image_id'] = old_new_id_mapping[old_id]

In [6]:
output_annotations = 'mapped_mapillary_1_annotations.json'
with open(output_annotations, 'w') as f:
    json.dump(anno_coco, f)

In [17]:
print(bldg_df.iloc[705,:])

Data collector                                                      Nirvik
Date                                                                   NaN
Country                                                          Hong Kong
City                                                             Hong Kong
Postal code                                                         999077
Latitude/Longitude                   22.314619105012405, 114.1881222653414
Image link                                                             NaN
Source                                                         Google Maps
Building category                           Engineered commercial building
Occupancy group                                            COM: Commerical
Occupancy type                                          COM1: Retail trade
Number of floors (excl. basement)                                6 or more
Basement                                                                No
Material                 

In [16]:
anno_coco['annotations'][31]

{'id': 32,
 'image_id': 705,
 'category_id': 1,
 'segmentation': [],
 'area': 998549.844,
 'bbox': [0.0, 0.0, 823.92, 1211.95],
 'iscrowd': 0,
 'attributes': {'category': 'Engineered commercial building',
  'occupancy_group': 'COM: Commerical',
  'occupancy_type': 'COM1: Retail trade',
  'no_floors': '6 or more',
  'basement': 'No',
  'material': 'Concrete',
  'roof_shape': 'Flat',
  'roof_covers': 'Built up roof',
  'shutters': 'No',
  'window_area': 'About 33% of surface',
  'occluded': False,
  'rotation': 0.0}}

In [23]:
# test=anno_coco['annotations'][3]

# test['attributes']['material'] = 'test'

In [24]:
anno_coco['annotations'][3]

{'id': 4,
 'image_id': 5,
 'category_id': 1,
 'segmentation': [],
 'area': 1424000.0,
 'bbox': [0.0, 0.0, 890.0, 1600.0],
 'iscrowd': 0,
 'attributes': {'category': 'Unknown',
  'occupancy_group': 'Unknown',
  'occupancy_type': 'Unknown',
  'no_floors': 'Unknown',
  'basement': False,
  'material': 'test',
  'roof_shape': 'Unknown',
  'roof_covers': 'Unknown',
  'shutters': False,
  'window_area': 'Unknown',
  'occluded': False,
  'rotation': 0.0}}

In [ ]:
# i just need to map annotationID to csvID to transfer attributes!